In [2]:
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box, \
    polygon_non_max_suppression, polygon_scale_coords
from utils.plots import colors, plot_one_box, polygon_plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

import subprocess as sp
import os

In [62]:
import numpy as np
# view_img=False,  # show results
# save_txt=False,  # save results to *.txt
# save_conf=False,  # save confidences in --save-txt labels
# ,  # save cropped prediction boxes
# nosave=False,  # do not save images/videos
# ,  # augmented inference
# update=False,  # update all models
# project='runs/detect',  # save results to project/name
# name='exp',  # save results to project/name
# exist_ok=False,  # existing project/name ok, do not increment
# line_thickness=3,  # bounding box thickness (pixels)
# hide_labels=False,  # hide labels
# hide_conf=False,  # hide confidences
# half=False,  # use FP16 half-precision inference

###################
augment=False
conf_thres=0.25
iou_thres=0.25
classes=None
agnostic_nms=False
max_det=1000 # maximum detections per image
device='2'
# weights='runs/train/exp52/weights/polygon_best.pt'
weights='runs/train/exp55/weights/polygon_last.pt'
imgsz=640
# source='testsample/0002.png'
###################

device = select_device(device)

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check image size

print(stride)

def detect(source):
    # Set Dataloader
    dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Run inference
    if device.type != 'cpu':
        with torch.no_grad():
            model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        pred = model(img, augment=augment)[0]

        # Apply polygon NMS
        pred = polygon_non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        results = []
        # Process detections
        for i, det in enumerate(pred):  # detections per image
    #         if webcam:  # batch_size >= 1
    #             p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(), dataset.count
    #         else:
            p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            gn = torch.tensor(im0.shape)[[1, 0, 1, 0, 1, 0, 1, 0]]  # normalization gain xyxyxyxy

            lines = []
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :8] = polygon_scale_coords(img.shape[2:], det[:, :8], im0.shape).round()
                # Print results
                
                # Write results
                for *xyxyxyxy, conf, cls in reversed(det):
                    xyxyxyxyn = (torch.tensor(xyxyxyxy).view(1, 8)).view(-1).tolist()  # normalized xyxyxyxy
                    line = (cls, *xyxyxyxyn, conf)  # label format
                    lines.append(line[1:-1])
#                     print(('%g ' * len(line)).rstrip() % line + '\n')
            results.append(lines)
        return np.array(results, dtype=np.int32)


32


In [25]:
detect('video_example2/0000.png').shape

1


(1, 1, 8)

In [ ]:
for path, img, im0s, vid_cap in dataset:
    print(path)

In [ ]:
%matplotlib inline
import cv2
import glob
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

show = False
save = True
# root = '/home/workspace/salim/data_set/20211123_1013_exposure_-10'
roots = glob.glob('/home/workspace/salim/data_set/*')

cnt = 0

for root in roots:
    
    root = '/home/workspace/salim/data_set/yuginong_banana'
    save_root = root.replace('data_set','data_set_crop_test')
#     if '50' in root:
#         continue
#     if os.path.isdir(save_root):
#         continue
    
    impath = glob.glob(root+'/*.png')
    impath = sorted(impath)

    # impath = glob.glob("runs/detect/exp65/*.png")
    cache = []
    for path in impath:
#         if cnt < 350:
#             cnt += 1
#             continue
    #     string = f"{path}"
    #     print(path)
        img = cv2.imread(path)

        anos = detect(path)[0]
        centers = []
        box_cnt = 0
        for ano in anos:
            ## edge에 가까운 것 제거
            if any(ano < 10):
                continue
            if any(ano[::2] > img.shape[1]-10):
                continue
            if any(ano[1::2] > img.shape[0]-10):
                continue

            point1, point2, point3, point4 = ano[:2], ano[2:4], ano[4:6], ano[6:]

            center = (point1+point2+point3+point4)/4
            center = center.astype(np.int32)
    #         string += f" {list(center)}"
    #         print(center)
            if show:
                cv2.line(img, point1,point1,(255,0,0),25)
                cv2.line(img, point2,point2,(255,127,0),25)
                cv2.line(img, point3,point3,(255,255,0),25)
                cv2.line(img, center,center,(0,255,255),25)

            new_w = int(np.linalg.norm(point1-point2))
            new_h = int(np.linalg.norm(point2-point3))

            ## 비슷한 위치의 라벨이 2개 이상일 때, 이상데이터 제거
            wh = float(new_w) + float(new_h)
            if len(anos) >= 2: #라벨 2개 이상
#                 print(sum(ano))
                sum_ano = sum(ano)
                duplicate = False
                for ano_ in anos:
                    if sum_ano == sum(ano_):
                        continue
                    if 0.95 * sum(ano_) < sum_ano and sum_ano < 1.05 * sum(ano_): # 비슷한 위치인가
                        duplicate = True
                        break
                if duplicate:
#                     print(sum(cache)/len(cache))
#                     print(wh)
                    if wh < 0.9 * sum(cache)/len(cache) or wh > 1.1 * sum(cache)/len(cache): # 이상 데이터면, 스킵(제거)
                        continue
                        
            centers.append(list(center))
            cache.append(wh)
            if len(cache) > 5:
                cache.pop(0)
            
            pts1 = np.float32([point1,point2,point3])
            pts2 = np.float32([[0,0], [new_w, 0], [new_w, new_h]])
            mat = cv2.getAffineTransform(pts1,pts2)
            dst = cv2.warpAffine(img, mat, None)
            dst = dst[:new_h,:new_w]

            if show:
                if len(anos) > 0:
                    fig = plt.figure()
                    fig.set_size_inches(10,10)
                    plt.imshow(img)
                    plt.show()
                    plt.imshow(dst)
                    plt.show()
            if save:
                os.makedirs(save_root, exist_ok=True)
                cv2.imwrite(os.path.join(save_root, os.path.basename(path).replace('.png',f"_{box_cnt}.png")),dst)
            box_cnt += 1

        result_dict = {}
        result_dict['path'] = path
        result_dict['centers'] = centers
        with open(save_root+'.txt',"a") as f:
            f.writelines(str(result_dict)+'\n')
        print(result_dict)
    break

In [50]:
1335.6/1267

1.0541436464088396

In [ ]:
cache = []
cache.append(1)
cache.append(2)
cache.append(3)
cache.pop()
cache

In [35]:
root

'/home/workspace/salim/data_set/20211123_1013_exposure_-10/'